In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('titanic_train').getOrCreate()

filepath = '/home/ubuntu/working/spark-examples/data/titanic_train.csv'
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True)

titanic_pdf = titanic_sdf.select('*').toPandas()



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/12 05:03:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
titanic_pdf.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


# Pandas DataFrame에서 데이터 조작하기|
- 컬럼에 대한 삽입과 수정을 쉽게 할 수 있다. `[ ]`를 사용해서..

In [3]:
import numpy as np

titanic_pdf_copy = titanic_pdf.copy()

# Fare에 10을 곱해서 Extra_Fare라는 컬럼에 집어넣기
titanic_pdf_copy['Extra_Fare'] = titanic_pdf_copy['Fare']*10
titanic_pdf_copy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Extra_Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,72.500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,712.833
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,79.250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,531.000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,80.500


In [4]:
# 컬럼 데이터 수정
titanic_pdf_copy['Fare'] = titanic_pdf_copy['Fare'] + 20

In [5]:
titanic_pdf_copy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Extra_Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,27.2500,None,S,72.500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,91.2833,C85,C,712.833
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,27.9250,None,S,79.250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,73.1000,C123,S,531.000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,28.0500,None,S,80.500


In [6]:
titanic_pdf_copy['Fare'] = titanic_pdf_copy['Fare'].astype(np.int64)
titanic_pdf_copy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Extra_Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,27,None,S,72.500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,91,C85,C,712.833
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,27,None,S,79.250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,73,C123,S,531.000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,28,None,S,80.500


# Spark Dataframe의 데이터 조작
- `withColumn()`: 기존 컬럼 값을 수정, 타입 변경, 신규 컬럼 추가
    - `withColumn('신규 또는 업데이트 되는 **컬럼명**','신규 또는 업데이트되는 값'`)
- 주의!! : 신규 또는 업데이트 되는 값을 생성 시에 기존 컬럼 값을 기반으로 한다면, 신규 컬럼은 **문자열**, 기존 컬럼은 반드시 컬럼객체(col('컬럼명'))을 사용
- 신규 컬럼을 추가하는 것은 `select` 로도 가능하다
- 컬럼명 변경은 `withColumnRename()` 메소드 사용 가능

In [7]:
from pyspark.sql.functions import col

titanic_sdf_copy  = titanic_sdf.select('*')

titanic_sdf_copy = titanic_sdf_copy.withColumn('Extra_Fare',col('Fare')*10)

In [8]:
titanic_sdf_copy.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Extra_Fare|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|      72.5|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   712.833|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     79.25|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     531.0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|    

In [9]:
# 기존 컬럼 Update
titanic_sdf_copy = titanic_sdf_copy.withColumn('Fare', col('Fare')+20)
titanic_sdf_copy.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Extra_Fare|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|  27.25| null|       S|      72.5|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|91.2833|  C85|       C|   712.833|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282| 27.925| null|       S|     79.25|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   73.1| C123|       S|     531.0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|  28.05| null|       S|    

In [10]:
titanic_sdf_copy = titanic_sdf_copy.withColumn('Fare', col('Fare').cast('Integer'))
titanic_sdf_copy.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|Extra_Fare|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|  27| null|       S|      72.5|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|  91|  C85|       C|   712.833|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  27| null|       S|     79.25|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|  73| C123|       S|     531.0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|  28| null|       S|      80.5|
+-----------+---

In [11]:
# pandas에서 상숫값으로 데이터를 삽입하거나 수정하기
titanic_pdf_copy['Extra_Fare'] = 10
titanic_pdf_copy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Extra_Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,27,None,S,10
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,91,C85,C,10
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,27,None,S,10
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,73,C123,S,10
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,28,None,S,10


In [12]:
# spark에서 상숫값으로 데이터를 삽입하거나 수정하기
titanic_sdf_copy = titanic_sdf_copy.withColumn('Extra_Fare',10) # 업데이트 할 값은 반드시 컬럼형이 되어야 한다. (ex. col('Fare')*10)
                                                                # 즉, spark는 broadcasting 연산이 일어나지 않는다.
titanic_sdf_copy.show(5)

TypeError: col should be Column

In [ ]:
# 상숫값으로 업데이트 하려면 반드시 lit 함수를 적용할 것
from pyspark.sql.functions import lit

titanic_sdf_copy = titanic_sdf_copy.withColumn('Extra_Fare', lit(10))
titanic_sdf_copy.show(5)

In [ ]:
titanic_sdf_copy = titanic_sdf_copy.withColumn('New_Column', lit('Test Column입니다'))
titanic_sdf_copy.show(5)

In [ ]:
from pyspark.sql.functions import substring

# select() 메소드를 이용해서 컬럼을 추가. 이 때 SQL의 substring 메소드 이용해 문자열의 일부를 추출하여 신규 컬럼생성.
titanic_sdf_copy = titanic_sdf_copy.select('*', col('Embarked').alias('E')) # select *, Embarked as E from titanic_sdf
titanic_sdf_copy.show(5)

In [ ]:
titanic_sdf_copy = titanic_sdf_copy.select('*', substring('Cabin', 0, 1).alias('Cabin_section'))

In [ ]:
titanic_sdf_copy.show(5)

In [ ]:
titanic_sdf_copy = titanic_sdf.select('*')
titanic_sdf_copy = titanic_sdf_copy.withColumn('탑승지', col('Embarked')).withColumn('Cabin_section', substring('Cabin',0,1))

In [ ]:
titanic_sdf_copy.show(5)
# 위에서 만든 '탑승지','Cabin_section' 컬럼이 잘 만들어져있따.;

In [ ]:
from pyspark.sql.functions import split

# Name 컬럼을 ','로 분리하여 Name1, Name2 생성

titanic_sdf_copy = titanic_sdf_copy.withColumn('first_name',split(col('Name'),',').getItem(0))
titanic_sdf_copy = titanic_sdf_copy.withColumn('last_name',split(col('Name'),',').getItem(1))

In [ ]:
titanic_sdf_copy.show(5)

In [ ]:
titanic_sdf_copy = titanic_sdf_copy.withColumnRenamed('탑승지','탑탑승지')
titanic_sdf_copy.show(5)

In [ ]:
titanic_sdf_copy = titanic_sdf_copy.withColumnRenamed('없는컬럼 넣어도','오류 안나는게 문제')
titanic_sdf_copy.show(5)

# Spark Dataframe의 컬럼, 로우(레코드) 삭제
- pandas의 데이터프레임은 `drop` 메소드를 사용.
- spark 데이터프레임에도 `drop` 이 존재. 컬럼만 삭제 가능
    - 여러개 삭제 시 `list` 사용 불가
- spark에서는 데이터 삭제가 원칙적으로는 없다.
    - 일반적으로 데이터란? 행(레코드)
    - 데이터 삭제가 없는 대신에 `filter` 를 이용해서 필요한 것만 추출해낸다.

In [ ]:
titanic_pdf_dropped = titanic_pdf.drop('Name',axis=1, inplace=False)
titanic_pdf_dropped.info() # Name컬럼이 사라진 것을 확인

In [ ]:
titanic_sdf_copy = titanic_sdf.select('*')
titanic_sdf_copy = titanic_sdf_copy.drop(col('Embarked')) # 컬럼명을 써도 되고, 안써도 됨.
titanic_sdf_copy.printSchema() # Embarked 컬럼이 사라진 것을 확인

In [ ]:
# 여러개의 컬럼을 한꺼번에 삭제하기. col을 쓰면 안됨
titanic_sdf_copy.drop('Fare','Cabin').printSchema()
titanic_sdf_copy.drop(col['Fare'],col('Cabin')).printSchema()

In [ ]:
# drop에 리스트로 컬럼명을 묶어서 삭제를 시도하면 오류
titanic_sdf_copy.drop(['Fare','Cabin']).printSchema()

In [ ]:
# 리스트 통해 컬럼 삭제하는 법? 애스터리스크(*) 이용
drop_columns = ['Fare','Cabin']
titanic_sdf_copy.drop(*drop_columns).printSchema()

In [ ]:
# 문자열 컬럼만 지우고싶을 때
drop_string_columns = [column_name for column_name, column_type in titanic_sdf_copy.dtypes if column_type =='string']

In [ ]:
print('drop columns :', drop_string_columns)

In [ ]:
titanic_sdf_copy.drop(*drop_string_columns).printSchema()

In [ ]:
titanic_sdf_copy.show(5)

In [ ]:
# Pclass = 1인 row를 삭제. 하지만 실제로는 삭제가 아닌,  pclass !=1인 데이터만 가져오는 것
titanic_sdf_removed_pclass_1 = titanic_sdf.filter(col('Pclass') != 1)
titanic_sdf_removed_pclass_1.show()

# Spark Dataframe의 dropna 사용하기

In [ ]:
# record에 하나라도 null 또는 NaN 값이 있으면 삭제한 결과 DataFrame이 반환.


print('dropna 이전: ', titanic_sdf.count())
titanic_sdf_dropna_1 = titanic_sdf.dropna()
print('dropna 이후: ', titanic_sdf_dropna_1.count())
# Null값, NaN값이 filtering된 것.

In [ ]:
print('dropna 이전: ', titanic_sdf.count())
titanic_sdf_dropna_2 = titanic_sdf.na.drop()
print('dropna 이후: ', titanic_sdf_dropna_2.count())
print('titanic_sdf.na의 타입: ', type(titanic_sdf.na))

# 모든 컬럼에 nan이 없는 데이터만 살아남은 것.

모든 컬럼에 NaN, None이 없어야지 필터링이 안된다. 즉, 모든 컬럼이 채워져있는 데이터만이 살아남을 수 있음.
```sql
select *
from titanic_sdf
where Name is not null and 
        age is not null and
        Embarked is not null ...
```

In [ ]:
# 특정 컬럼을 지정하여 거기에 Null이 있는 경우에만 삭제
titanic_sdf_dropna_3 = titanic_sdf.na.drop(subset = ['Age','Embarked'])
print('dropna 이후: ', titanic_sdf_dropna_3.count())

# Age, Embarked 둘 다 null이 아닌 레코드 숫자

In [15]:
# Null이 아닌 Age와 Embarked의 개수
# select count(*) from titanic_sdf where age is not null and embarked is not null

# col(c).isNotNull 
titanic_sdf.filter(col('Age').isNotNull() & col('Embarked').isNotNull()).count()



712

In [16]:
# dropna() 메소드를 로직으로 구현. 
where_str = ''
column_count = len(titanic_sdf.columns)
for index, column_name in enumerate(titanic_sdf.columns):
    where_str += (column_name +' IS NOT NULL ') 
    if index < column_count - 1:
        where_str += 'and '
print(where_str)

PassengerId IS NOT NULL and Survived IS NOT NULL and Pclass IS NOT NULL and Name IS NOT NULL and Sex IS NOT NULL and Age IS NOT NULL and SibSp IS NOT NULL and Parch IS NOT NULL and Ticket IS NOT NULL and Fare IS NOT NULL and Cabin IS NOT NULL and Embarked IS NOT NULL 


In [17]:
titanic_sdf.filter(where_str).count()

183

- numpy (NUMerical PYthon)
    - None: 객체가 없을 때 
    - NaN: Not a Number, 숫자가 없을 때 
- spark
    - Null: None을 Null로 처리. 
    - NaN: 숫자가 없는 경우
    - Pandas DF -> Spark DF로 변환시
        - spark 3.0이하: Nan ->Nan
        - spark 3.0 이후: Nan -> Null
        - **혹시라도 df에 nan값이 들어있으면 null값으로 바꿔줘라(isnull을 쓸 수 있게)**

# Pandas 와 Spark에서의 None, Null, NaN

In [20]:
a = None
print(type(a), a)

<class 'NoneType'> None


- titanic pandas => string아닌 object타입
    - object -> None
    - int -> NaN
- titanic spark df => string타입
    - 전부다 null값

In [21]:
titanic_sdf.show(10)
titanic_pdf.head(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C


In [24]:
import pandas as pd
import numpy as np

# none은 object형 array에만 넣을 수 있다.
array = np.array([0,1,2,'hello',None])
print(array, array.dtype)

[0 1 2 'hello' None] object


In [25]:
# 숫자 형식의 array는 None이 NaN으로 바뀐다.
array = np.array([0,1,2,None], dtype=np.float32)
print(array, array.dtype)

[ 0.  1.  2. nan] float32


In [28]:
print(titanic_pdf['Cabin'].to_numpy().dtype) #None - object

object


In [29]:
print(titanic_pdf['Age'].to_numpy().dtype) # NaN - float 64

float64


In [ ]:
# spark 3.2버전에서는 pandas DataFrame으로 spark dataframe을 생성해도 NaN, None 모두 null로 변환


# Spark Dataframe에서 Null과 NaN 찾기
- pandas의 isnull(), isna()는 서로 동일한 메소드로서 모두 None과 NaN을 찾는다.
- spark의 isnull(), isna()는 서로 다른 것을 찾음.
    - isnull(): None을 찾음 (isnull 쓰는게 훨 좋음)
    - isna(): NaN을 찾음
    

In [31]:
# Pandas
print(titanic_pdf[['Age','Cabin']].head(10))
print(titanic_pdf[['Age','Cabin']].isna()) # pandas는 isnull() = isna()
print(titanic_pdf[['Age','Cabin']].isnull())

    Age Cabin
0  22.0  None
1  38.0   C85
2  26.0  None
3  35.0  C123
4  35.0  None
5   NaN  None
6  54.0   E46
7   2.0  None
8  27.0  None
9  14.0  None
       Age  Cabin
0    False   True
1    False  False
2    False   True
3    False  False
4    False   True
..     ...    ...
886  False   True
887  False  False
888   True   True
889  False  False
890  False   True

[891 rows x 2 columns]
       Age  Cabin
0    False   True
1    False  False
2    False   True
3    False  False
4    False   True
..     ...    ...
886  False   True
887  False  False
888   True   True
889  False  False
890  False   True

[891 rows x 2 columns]


In [34]:
# Spark에서 isNull()은 컬럼 형식에서만 사용이 가능.
# isnan은 pyspark.sql.functions에서 불러와야지만 사용가능
from pyspark.sql.functions import col, isnan

titanic_sdf.filter(col('Age').isNull()).show(10)
titanic_sdf.filter(isnan(col('Age'))).show(10) # nan값이 없어서 조회 안됨


+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|  330877|  8.4583| null|       Q|
|         18|       1|     2|Williams, Mr. Cha...|  male|null|    0|    0|  244373|    13.0| null|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|    2649|   7.225| null|       C|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|    2631|   7.225| null|       C|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|  330959|  7.8792| null|       Q|
|         30|       0|     3| Todoroff, Mr. Lalio|  male|null|    0|    0|  349216|  7.8958| null|       S|
|         32|       1|     1

# Null이 있는 컬럼명과 Null 개수 찾기

In [41]:
titanic_sdf.select([col(column_name).isNull() for column_name in titanic_sdf.columns]).show()
titanic_sdf.select([col(column_name).isNull().alias(column_name) for column_name in titanic_sdf.columns]).show()

+---------------------+------------------+----------------+--------------+-------------+-------------+---------------+---------------+----------------+--------------+---------------+------------------+
|(PassengerId IS NULL)|(Survived IS NULL)|(Pclass IS NULL)|(Name IS NULL)|(Sex IS NULL)|(Age IS NULL)|(SibSp IS NULL)|(Parch IS NULL)|(Ticket IS NULL)|(Fare IS NULL)|(Cabin IS NULL)|(Embarked IS NULL)|
+---------------------+------------------+----------------+--------------+-------------+-------------+---------------+---------------+----------------+--------------+---------------+------------------+
|                false|             false|           false|         false|        false|        false|          false|          false|           false|         false|           true|             false|
|                false|             false|           false|         false|        false|        false|          false|          false|           false|         false|          false|          

In [42]:
from pyspark.sql.functions import count, when

titanic_sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [43]:
titanic_sdf.select([count(when(col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



```python
col(c).isNull()이 true인 경우
-> when(col(c).isNull(), c)
-> count(c)
```

# Null값 처리하기

In [44]:
# Pandas Dataframe에서 Age의 NaN값을 나이의 평균으로 채우기
titanic_pdf['Age'] = titanic_pdf['Age'].fillna(titanic_pdf['Age'].mean(), inplace=False)
titanic_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int32  
 1   Survived     891 non-null    int32  
 2   Pclass       891 non-null    int32  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int32  
 7   Parch        891 non-null    int32  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int32(5), object(5)
memory usage: 66.3+ KB


In [45]:
# 스파크 데이터프레임에서 결측치(Null)을 채우는 방법

# 1. fillna(숫자 데이터에만)
titanic_sdf.fillna(value=999).show(10) # 숫자형 컬럼에 nan값에 999라고 채워짐

# 2. na.fill
titanic_sdf.na.fill(value=999).show(10)

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|999.0|    0|   

In [46]:
titanic_sdf.fillna(value='NA').show(10) # 이젠 Age에는 안채워지고 Cabin에만 채워짐

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|   NA|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|   NA|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|   NA|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

숫자형식의 데이터가 들어가면 숫자 컬럼에만 결측치가 채워지고, 문자열 형식의 데이터가 들어가면 문자열 컬럼에만 결측치가 채워진다.

In [48]:
# subset을 이용해서 특정 컬럼에만 Null값 처리하기
titanic_sdf.fillna(value=999, subset=['Age']).show(10)

titanic_sdf.fillna(value='NA', subset=['Cabin']).show(10)

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|999.0|    0|   

In [49]:
# Age에 대한 결측치 처리를 Age의 평균으로 하기
from pyspark.sql.functions import avg, col
import pyspark.sql.functions as F

avg_age = titanic_sdf.select(F.avg(F.col('Age')))
avg_age.show()
type(avg_age)


+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



pyspark.sql.dataframe.DataFrame

(Row(avg(Age)=29.69911764705882), pyspark.sql.types.Row)

In [50]:
# value 인자로 데이터프레임이 들어가면 오류 발생
titanic_sdf.fillna(value=avg_age, subset=['Age'])

TypeError: value should be a float, int, string, bool or dict

In [65]:
avg_age_row = avg_age.first()
avg_age_row, type(avg_age_row)

(Row(avg(Age)=29.69911764705882), pyspark.sql.types.Row)

In [66]:
avg_age_value = avg_age_row[0]
avg_age_value, type(avg_age_value)

(29.69911764705882, float)

In [67]:
titanic_sdf.fillna(value= avg_age_value, subset=['Age']).show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

In [68]:
spark.stop()